# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [506]:
from typing import Tuple, List
import copy
import random
import math
import time

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState

Wielkość planszy

In [507]:
ROW_COUNT = 4
COLUMN_COUNT = 7

In [508]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game
        self.fields = game.state.fields
        self.columns_count = len(self.fields) #Trzeba się niezwykle napracować nad taką głupotą
        self.rows_count = len(self.fields[0])
        # print(self.rows_count)
        # print(self.columns_count)
        self.p_min = game.get_players()[0]
        self.p_max = game.get_players()[1]

    def evaluate_position(self, player: Player)->float:
        
        # game1=copy.deepcopy(self.game)
        pass

    def minimax(self, depth, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""

        if depth == 0:
            return [None, 1]
        elif self.get_wining_move()[0] != -1:
            # print("ruch wy")
            
            if is_maximizing_player:
                return [self.get_wining_move()[0], 1000]
            else:
                return [self.get_wining_move()[0], -1000]
                                
        tab_h = []
        if self.game.get_moves() != None:
            for m in self.game.get_moves():

                game_c=copy.deepcopy(self.game)
                game_c.make_move(m)
                mmsol = MinMaxSolver(game_c)

                # print(game_c)
                # print(depth)

                best_mov = mmsol.minimax(depth-1,(not is_maximizing_player))
                # print(best_mov)
                if best_mov[0] != None:
                    tab_h.append([m.column, best_mov[1]])
                else:
                    tab_h.append([None, best_mov[1]])
                # tab_h.append([m.column, best_mov[1]])

            # print(depth)              
            # print(tab_h)
            best_moves_without_None = []
            for mov in tab_h:
                if mov[0] != None:
                    best_moves_without_None.append(mov)
            # print(best_moves_without_None)
            if len(best_moves_without_None) == 0:
                #w tym wypadku musi być heurystyka
                return [None, None]
            else:
                if is_maximizing_player:
                    #max wartosc z drugiej pozycji krotki
                    # print("wybieram max")
                    best_moves_without_None.sort(key=lambda a: a[1], reverse=True)

                else:
                    # print("wybieram min")
                    best_moves_without_None.sort(key=lambda a: a[1], reverse=False)               
            
            # if is_maximizing_player:
            #     #max wartosc z drugiej pozycji krotki
            #     print("wybieram max")
            #     tab_h.sort(key=lambda a: a[1], reverse=True)

            # else:
            #     print("wybieram min")
            #     tab_h.sort(key=lambda a: a[1], reverse=False)

                return best_moves_without_None[0]
        else:
            return [None, 0]
        

    def get_wining_move(self):#returns -1 if wining move do not exist

        moves = self.game.get_moves()
        for m in moves:
            game1=copy.deepcopy(self.game)
            player1 = game1.get_current_player()
            game1.make_move(m)
            # print(game1)
            player2= game1.get_current_player()
            if(game1.get_winner()==player1):
                return [m.column, player1]
            elif(game1.get_winner()==player2):
                return [m.column, player2]
        return [-1, None]

    def get_column_height(self, col: int) -> int:
        i=0
        for i, field in enumerate(self.fields[col]):
            if field is not None:
                i = i + 1
            else:
                return i
                
    def heuristic(self, x, y):
        y_score = math.floor((self.columns_count)/2)-math.floor(abs(y-(self.rows_count-1)/2))
        x_score = math.floor((self.rows_count)/2)-math.floor(abs(x-(self.columns_count-1)/2))
        return x_score+y_score
            


In [509]:
class ConnectFourGame:
    def __init__(self, game: ConnectFour):
        self.game = game

    def Bot_vs_Bot_minmax(self):
        
        depth = 2
        if(self.game.get_current_player() == self.game.first_player):
            print("zaczyna a")
            a_turn = True
        else:
            print("zaczyna b")
            a_turn = False

        print("Pozycja początkowa")
        print(self.game)

        while(self.game.get_moves() != None and self.game.get_winner() == None):
            self.game.make_move(ConnectFourMove(self.move(depth,0,0,a_turn,True)))
            a_turn = not a_turn
            print(self.game)
            time.sleep(1)

        if(self.game.get_winner()==self.game.first_player):
            print("wygrał a")
        elif(self.game.get_winner()!=None):
            print("wygrał b")
        else:
            print("remis")   
    
    
    def move(self, depth, alpha:float, beta:float, is_maximizing_player:bool, minmax_algorithm:bool)-> int:
        mmsol = MinMaxSolver(self.game)
        if minmax_algorithm:
            best_mov = mmsol.minimax(depth, is_maximizing_player)
        else:
            best_mov = mmsol.minimax(depth,alpha,beta, is_maximizing_player)

        
        if best_mov[1] == None:
            print("heu heu")
            moves = self.game.get_moves()
            tab_h = []
            for m in moves:
                tab_h.append([m.column, mmsol.heuristic(m.column,mmsol.get_column_height(m.column))])

            if(len(tab_h) == 0): #Warunek remisu
                return None

            # print(tab_h)
            tab_h.sort(key=lambda a: a[1], reverse=True)
            # print(tab_h)
            h_max = tab_h[0][1]
            tab_h_max = []
            for h in tab_h:
                if h[1] == h_max:
                    tab_h_max.append(h[0])
            # print(tab_h_max)
            rand_h = random.choice(tab_h_max)

            return rand_h
        else:
            return best_mov[0]

    


In [510]:
#trasz
ROW_COUNT = 4
COLUMN_COUNT = 5
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))

# print(game1)
# minmax = MinMaxSolver(game1)
# print(minmax.get_wining_move())
# x = minmax.move(4,0,0,False)
# print("Opt move")
# print(x)

In [511]:
#BvB test
ROW_COUNT = 5
COLUMN_COUNT = 5
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

#Test 1
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
#Test 2
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(3))
# game1.make_move(ConnectFourMove(1))
# game1.make_move(ConnectFourMove(2))
#Test3
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))
game1.make_move(ConnectFourMove(3))
game1.make_move(ConnectFourMove(4))
game1.make_move(ConnectFourMove(2))
game1.make_move(ConnectFourMove(3))
game1.make_move(ConnectFourMove(3))
game1.make_move(ConnectFourMove(4))
game1.make_move(ConnectFourMove(4))

# minmax = MinMaxSolver(game1)
# minmax.Bot_vs_Bot_Game(8,0,0,True)
# minmax.move(2,0,0,False)
Conn_Game = ConnectFourGame(game1)
Conn_Game.Bot_vs_Bot_minmax()

zaczyna b
Pozycja początkowa
Current player: b
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][a][a]
[ ][ ][a][b][b]
[ ][a][b][a][b]
[4, 1000]
[4, 1000]
[4, 1000]
[4, 1000]
[None, 1]
[None, 1]
[None, 1]
[None, 1]
[None, 1]
[None, None]
Current player: a
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ]
[ ][ ][ ][a][a]
[ ][ ][a][b][b]
[b][a][b][a][b]
Current player: b
[ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a]
[ ][ ][ ][a][a]
[ ][ ][a][b][b]
[b][a][b][a][b]
wygrał a


TEST HEURYSTYKI

In [512]:
ROW_COUNT = 8
COLUMN_COUNT = 10
p1 = Player("a")
p2 = Player("b")
t = [ [0]*8 for i in range(10)]
game_0 =ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
minmax_0 = MinMaxSolver(game_0)
for i in range(COLUMN_COUNT):
    for j in range(ROW_COUNT):
        t[i][j] = minmax_0.heuristic(i,j)
print(t)
print(t[9][0])

# ROW_COUNT = 4
# COLUMN_COUNT = 4
# p1 = Player("a")
# p2 = Player("b")
# t = [ [0]*4 for i in range(4)]
# game_0 =ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
# minmax_0 = MinMaxSolver(game_0)
# for i in range(COLUMN_COUNT):
#     for j in range(ROW_COUNT):
#         t[i][j] = minmax_0.heuristic(i,j)
# print(t)



[[2, 3, 4, 5, 5, 4, 3, 2], [3, 4, 5, 6, 6, 5, 4, 3], [4, 5, 6, 7, 7, 6, 5, 4], [5, 6, 7, 8, 8, 7, 6, 5], [6, 7, 8, 9, 9, 8, 7, 6], [6, 7, 8, 9, 9, 8, 7, 6], [5, 6, 7, 8, 8, 7, 6, 5], [4, 5, 6, 7, 7, 6, 5, 4], [3, 4, 5, 6, 6, 5, 4, 3], [2, 3, 4, 5, 5, 4, 3, 2]]
2


In [513]:

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))


print(game.get_winner() == p1)
print(game.get_winner() == p2)
game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
print(game.get_winner() == p1)
print(game.get_winner() == p2)


moves = game.get_moves()
fields = game.state.fields
m1 = moves[0]
print(m1.column)
print(len(moves))
print("col height check")

print(game)

False
False
True
False
0
10
col height check
Current player: b
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ][ ][ ][ ]
[ ][ ][a][a][b][ ][ ][ ][ ][ ]


Rozgrywka

In [514]:
ROW_COUNT = 4
COLUMN_COUNT = 4
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(0))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(1))

minmax = MinMaxSolver(game1)
# print(game1)
# print(minmax.get_column_height(1))
# print(minmax.get_wining_move())

# sample = [('Jack', 76), ('Beneth', 78), ('Cirus', 77), ('Faiz', 79)]
# sample.sort(key=lambda a: a[1], reverse=True)
# print(sample)

# for m in game1.get_moves():

#             game_c=copy.deepcopy(game1)
#             game_c.make_move(m)
#             print(game_c)
        

# print(game1)
x = minmax.minimax(2,0,0,True)
print(x)

TypeError: MinMaxSolver.minimax() takes 3 positional arguments but 5 were given